In [ ]:
# =============================================
# UTKFace Age & Gender Prediction (ResNet18)
# =============================================

!pip install torch torchvision opencv-python tqdm

import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from tqdm import tqdm

In [ ]:
# ==========================
# 1. Dataset Class
# ==========================
class UTKFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(".jpg")]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        path = os.path.join(self.root_dir, img_name)
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Parse labels
        age, gender, *_ = img_name.split("_")
        age = int(age)
        gender = int(gender)

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(age, dtype=torch.float32), torch.tensor(gender, dtype=torch.long)

In [ ]:
# ==========================
# 2. Transformations
# ==========================
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),   # ResNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Imagenet normalization
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# ==========================
# 3. Load Dataset
# ==========================

dataset_path = "/content/drive/MyDrive/Age_Gender_Detection/UTKFace_5k"

dataset = UTKFaceDataset(dataset_path, transform=transform)

# Train-test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Total: {len(dataset)}, Train: {len(train_dataset)}, Test: {len(test_dataset)}")

In [ ]:
# ==========================
# 4. ResNet18 Model with Two Heads
# ==========================
resnet18 = models.resnet18(pretrained=True)

# Freeze all layers
for param in resnet18.parameters():
    param.requires_grad = False

# Unfreeze last block for fine-tuning
for param in resnet18.layer4.parameters():
    param.requires_grad = True

class AgeGenderResNet(nn.Module):
    def __init__(self, backbone):
        super(AgeGenderResNet, self).__init__()
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # remove final fc
        in_features = backbone.fc.in_features  # 512 for resnet18

        # Shared fully connected layer
        self.fc = nn.Linear(in_features, 256)

        # Heads
        self.age_head = nn.Linear(256, 1)        # Regression
        self.gender_head = nn.Linear(256, 2)     # Classification

    def forward(self, x):
        x = self.backbone(x)          # [B, 512, 1, 1]
        x = x.view(x.size(0), -1)     # flatten
        x = F.relu(self.fc(x))

        age = self.age_head(x)
        gender = self.gender_head(x)
        return age, gender


In [ ]:
# ==========================
# 5. Training Setup
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AgeGenderResNet(resnet18).to(device)

criterion_age = nn.MSELoss()
criterion_gender = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005)

In [ ]:
# ==========================
# 6. Training Loop
# ==========================
EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for imgs, ages, genders in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, ages, genders = imgs.to(device), ages.to(device), genders.to(device)

        optimizer.zero_grad()
        age_pred, gender_pred = model(imgs)

        loss_age = criterion_age(age_pred.squeeze(), ages)
        loss_gender = criterion_gender(gender_pred, genders)
        loss = loss_age + loss_gender

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

In [ ]:
# ==========================
# 7. Evaluation
# ==========================
model.eval()
correct_gender, total = 0, 0
age_mae = 0

with torch.no_grad():
    for imgs, ages, genders in test_loader:
        imgs, ages, genders = imgs.to(device), ages.to(device), genders.to(device)
        age_pred, gender_pred = model(imgs)

        # Gender accuracy
        predicted_gender = torch.argmax(gender_pred, dim=1)
        correct_gender += (predicted_gender == genders).sum().item()
        total += genders.size(0)

        # Age error (MAE)
        age_mae += torch.abs(age_pred.squeeze() - ages).sum().item()

gender_acc = 100 * correct_gender / total
age_mae = age_mae / total

print(f"✅ Gender Accuracy: {gender_acc:.2f}%")
print(f"✅ Age MAE: {age_mae:.2f} years")

In [ ]:
# ==========================
# 8. Save Model
# ==========================
torch.save(model.state_dict(), "age_gender_resnet18.pth")
print("Model saved as age_gender_resnet18.pth")